# libraries


In [12]:
!pip install plotly pandas numpy dash dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 14.8 MB/s eta 0:00:00


In [3]:
!pip install plotly dash cufflinks


In [4]:
!pip install chart_studio # Install the missing 'chart_studio' package

In [5]:
!pip install --upgrade numba

In [6]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import dcc, html, Input, Output
import plotly.express as px
from dash.dependencies import Input, Output
import cufflinks as cf
import chart_studio.plotly as py
import warnings
warnings.filterwarnings('ignore')

<ipython-input-6-f6fb2468d7d0>:9: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-6-f6fb2468d7d0>:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Data loading

In [7]:
df = pd.read_csv('/content/owid-covid-data.csv')

In [8]:
df


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344774,ZWE,Africa,Zimbabwe,2023-09-23,265748.0,0.0,0.0,5718.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
344775,ZWE,Africa,Zimbabwe,2023-09-24,265748.0,0.0,0.0,5718.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
344776,ZWE,Africa,Zimbabwe,2023-09-25,265748.0,0.0,0.0,5718.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
344777,ZWE,Africa,Zimbabwe,2023-09-26,265748.0,0.0,0.0,5718.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


# Data Pre-processing

```



In [11]:
df['date'] = pd.to_datetime(df['date'])

df.sort_values('date', inplace=True)

df = df.drop_duplicates(subset=['date'])

missing_values = df.isnull().sum()
print("🔹 Missing Values:\n", missing_values[missing_values > 0])

df.fillna(method='ffill', inplace=True)  # Forward fill

# Answer 1: Time Range of the Dataset
start_date, end_date = df['date'].min(), df['date'].max()
print(f"🔹 Data spans from {start_date} to {end_date}")

# Answer 2: Plot Time Series Trend
fig_trend = px.line(df, x='date', y='total_cases', title="📈 Overall Time Series Trend")
fig_trend.show()

# Answer 3: Distribution of Data Points Over Time
fig_hist = px.histogram(df, x='date', nbins=30, title="🗓 Distribution of Data Points Over Time")
fig_hist.show()

# Answer 4: Detecting Outliers (Z-score Method)
# Assuming 'total_cases' is the relevant numeric column for outlier detection:
df['z_score'] = (df['total_cases'] - df['total_cases'].mean()) / df['total_cases'].std()
df_outliers = df[df['z_score'].abs() > 3]  # Outliers (Z-score > 3)

fig_outliers = px.scatter(df, x='date', y='total_cases', title="🚨 Outliers in Time Series", color=df['z_score'].abs() > 3)
fig_outliers.show()

# Answer 5: Rolling Average for Smoother Trends
# Assuming 'total_cases' is the relevant numeric column for rolling average:
df['rolling_avg'] = df['total_cases'].rolling(window=7).mean()
fig_rolling = px.line(df, x='date', y='rolling_avg', title="📊 7-Day Rolling Average")
fig_rolling.show()

# Answer 6: Weekday vs Weekend Trends
# Assuming 'total_cases' is the relevant numeric column for weekday/weekend analysis:
df['weekday'] = df['date'].dt.day_name()
fig_weekday = px.box(df, x='weekday', y='total_cases', title="📅 Weekday vs Weekend Trends", color='weekday')
fig_weekday.show()

# Answer 7: Monthly Seasonality Trends
# Assuming 'total_cases' is the relevant numeric column for monthly seasonality:
df['month'] = df['date'].dt.strftime('%B')
fig_monthly = px.box(df, x='month', y='total_cases', title="📆 Monthly Seasonality", color='month')
fig_monthly.show()

# Answer 8: Correlation with Other Variables (If available)
if 'other_variable' in df.columns:
    fig_corr = px.scatter(df, x='value', y='other_variable', title="🔗 Correlation Between Value and Other Variable")
    fig_corr.show()

print("✅ Time Series Analysis Completed!")


🔹 Missing Values:
 total_cases                                     8
new_cases                                       2
new_cases_smoothed                              7
total_deaths                                    8
new_deaths                                      2
new_deaths_smoothed                             7
total_cases_per_million                         8
new_cases_per_million                           2
new_cases_smoothed_per_million                  7
total_deaths_per_million                        8
new_deaths_per_million                          2
new_deaths_smoothed_per_million                 7
reproduction_rate                              59
icu_patients                                   69
icu_patients_per_million                       69
hosp_patients                                  69
hosp_patients_per_million                      69
weekly_icu_admissions                         174
weekly_icu_admissions_per_million             174
weekly_hosp_admissions         

✅ Time Series Analysis Completed!


# 📊 Advanced Exploratory Data Analysis (EDA) & Interactive Dashboard using Plotly

In [15]:

df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', inplace=True)

df.fillna(method='ffill', inplace=True)

df['day_of_week'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month_name()
df['year'] = df['date'].dt.year
df['rolling_avg'] = df['total_cases'].rolling(window=7).mean()

Q1 = df['total_cases'].quantile(0.25)
Q3 = df['total_cases'].quantile(0.75)
IQR = Q3 - Q1
df['outlier'] = (df['total_cases'] < (Q1 - 1.5 * IQR)) | (df['total_cases'] > (Q3 + 1.5 * IQR))

# --- DASHBOARD CREATION ---
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = dbc.Container([
    html.H1("📊 Time Series Data Dashboard", className="text-center mt-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(id="time_series_chart"), width=12),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="rolling_avg_chart"), width=6),
        dbc.Col(dcc.Graph(id="outliers_chart"), width=6),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="weekday_trends"), width=6),
        dbc.Col(dcc.Graph(id="monthly_trends"), width=6),
    ]),

], fluid=True)

# Callbacks to Update Graphs
@app.callback(
    Output("time_series_chart", "figure"),
    Output("rolling_avg_chart", "figure"),
    Output("outliers_chart", "figure"),
    Output("weekday_trends", "figure"),
    Output("monthly_trends", "figure"),
    Input("time_series_chart", "id")
)
def update_graphs(_):

    fig_time_series = px.line(df, x='date', y='total_cases', title="📈 Time Series Trend", markers=True)


    fig_rolling_avg = px.line(df, x='date', y='rolling_avg', title="📊 7-Day Rolling Average")

    fig_outliers = px.scatter(df, x='date', y='total_cases', color=df['outlier'].map({True: 'Outlier', False: 'Normal'}),
                              title="🚨 Outliers Detection (IQR Method)")


    fig_weekday = px.box(df, x='day_of_week', y='total_cases', title="📅 Weekday Trends", color='day_of_week')


    fig_monthly = px.box(df, x='month', y='total_cases', title="📆 Monthly Trends", color='month')

    return fig_time_series, fig_rolling_avg, fig_outliers, fig_weekday, fig_monthly

# Run App
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>